# Invoke Model Endpoint:

## 1. Client to Endpoint using Sagemaker SDK

Previous activity: completing the model training and model endpoint deployment.

In [1]:
import boto3
import sagemaker
import pandas as pd

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [2]:
# Start a sagemaker session with "user"
boto3_session = boto3.Session(
    profile_name="user", region_name="ap-southeast-1")
sess = sagemaker.Session(boto_session=boto3_session)

In [3]:
# Create the predictor
predictor = sagemaker.predictor.Predictor(
    endpoint_name="xgboost-loan", sagemaker_session=sess
)
predictor.serializer = CSVSerializer()

In [4]:
# Prepare the input data sample
input_sample_1 = "1.0, 1.0, 0.0, 1.0, 0.0, 2083.0, 3150.0, 128.0, 360.0, 1.0, 0.0, 1.0, 0.0"  # true value: 1
input_sample_2 = "1.0, 0.0, 0.0, 0.0, 0.0, 2333.0, 1451.0, 102.0, 480.0, 0.0, 0.0, 0.0, 1.0"  # true value: 0
input_sample_3 = "1.0, 1.0, 1.0, 1.0, 0.0, 6325.0, 0.0, 175.0, 360.0, 1.0, 0.0, 1.0, 0.0"  # true value: 1

In [5]:
# Invoke the prediction
pred = predictor.predict([input_sample_1, input_sample_2, input_sample_3])
pred.decode("utf-8")

'1.0,0.0,1.0'

In [6]:
# Load test data
test_data = pd.read_csv("D:/3. Projects/AWS Sagemaker/others/loan_test_data.csv")
print(f"The length of the test data is {len(test_data)}")

The length of the test data is 96


In [7]:
# Load test data
test_input = []
with open("D:/3. Projects/AWS Sagemaker/others/loan_test_data.csv", 'r') as f:
    f.readline()
    test_input = f.readlines()

In [8]:
# Batch Prediction
def transform(data):
    feature = data.split(',')

    if feature[1] == "Male":
        feature[1] = "1"
    else:
        feature[1] = "0"

    if feature[2] == "Yes":
        feature[2] = "1"
    else:
        feature[2] = "0"

    if feature[3] == "3+":
        feature[3] = "3"
    else:
        feature[3] = feature[3]

    if feature[4] == "Graduate":
        feature[4] = "1"
    else:
        feature[4] = "0"

    if feature[5] == "Yes":
        feature[5] = "1"
    else:
        feature[5] = "0"

    feature.extend(["0", "0", "0"])

    if feature[11] == "Rural":
        feature[12] = "1"
    elif feature[11] == "Semiurban":
        feature[13] = "1"
    elif feature[11] == "Urban":
        feature[14] = "1"
    feature.pop(11)

    feature = ','.join(feature[1:])

    return feature


def batch_prediction(test_input, batch_size):
    predictions = []

    # Transform input data
    test_input = [transform(row.strip()) for row in test_input]

    for i in range(0, len(test_input), batch_size):
        print(f"predicting row {i}")

        # Predict the test data
        pred = predictor.predict(test_input[i:batch_size+1])
        pred = pred.decode("utf-8")
        pred = pred.split(',')
        predictions += pred

    return predictions

In [10]:
# Batch prediction on the test data
pred = batch_prediction(test_input, 96)
pred

predicting row 0


['1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0']

In [11]:
# Number of predictions
print(f"Number of predictions: {len(pred)}")

Number of predictions: 96


Finally, delete the endpoint (if not used anymore)